In [1]:
import torch
from torch.utils.data import SubsetRandomSampler

from src.dataset import ImageDataset
from src.utils import get_indices, FFTloss
from src.mwcnn import MWCNN
from src.training import Trainer
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Dataset part used for testing
TEST_SPLIT = 0.2
# Batch size for training. Limited by GPU memory
BATCH_SIZE = 6
# Dataset folder used
DATASETS = ['e9_5_GLM87a_cycle1_8_8', 'e12_5_slide7_round1_section1']
ROOTDIR = 'data\mip2edof_2samples'
# Training Epochs
EPOCHS = 100

c:\Users\jurri\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
c:\Users\jurri\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback

## 1 Initiate train and test loader

In [2]:
image_dataset = ImageDataset(ROOTDIR, DATASETS)

train_indices, test_indices = get_indices(len(image_dataset), image_dataset.root_dir, TEST_SPLIT)
train_sampler, test_sampler = SubsetRandomSampler(train_indices), SubsetRandomSampler(test_indices)

trainloader = torch.utils.data.DataLoader(image_dataset, BATCH_SIZE, sampler=train_sampler)
testloader = torch.utils.data.DataLoader(image_dataset, 1, sampler=test_sampler)

c:\Users\jurri\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\transforms\v2\_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `transforms.Compose([transforms.ToImageTensor(), transforms.ConvertImageDtype()])`.
  warnings.warn(


## 1.1 Show example of images

In [3]:
# import matplotlib.pyplot as plt

# fig, ax = plt.subplots(nrows=6, ncols=2, figsize=(20, 60))
# for data in trainloader:
#     input_image = data['input_image'].squeeze().permute(1, 2, 0)
#     output_image = data['output_image'].squeeze().permute(1, 2, 0)
#     for i in range(input_image.shape[2]):
#         ax[i,0].imshow(input_image[:,:,i], cmap='gray', vmin=0, vmax=16383, aspect='equal')
#         ax[i,1].imshow(output_image[:,:,i], cmap='gray', vmin=0, vmax=16383, aspect='equal')
#     break

# plt.tight_layout()

## 2 Initiate model

In [6]:
MWCNN_model = MWCNN(n_feats=16 ,n_colors=1).to(device)

## 3. Model Training and Testing

In [7]:
# Training
criterion = torch.nn.L1Loss()
MWCNN_trainer = Trainer(MWCNN_model, criterion, device)

loss_record = MWCNN_trainer.train(EPOCHS,trainloader,mini_batch=5)

print(f'Training finished!')

Starting Training Process
Batch: 05,	Batch Loss: 22.8190264
Batch: 10,	Batch Loss: 15.1492241
Batch: 15,	Batch Loss: 14.9627145
Epoch: 001,  Loss:17.6436550,  
Batch: 05,	Batch Loss: 17.3519298
Batch: 10,	Batch Loss: 13.0432203
Batch: 15,	Batch Loss: 18.5730960
Epoch: 002,  Loss:16.3227487,  
Batch: 05,	Batch Loss: 14.4669629
Batch: 10,	Batch Loss: 18.6732505
Batch: 15,	Batch Loss: 14.3683130
Epoch: 003,  Loss:15.8361755,  
Batch: 05,	Batch Loss: 19.1413943
Batch: 10,	Batch Loss: 14.3565661
Batch: 15,	Batch Loss: 13.9366449
Epoch: 004,  Loss:15.8115351,  
Batch: 05,	Batch Loss: 16.1203888
Batch: 10,	Batch Loss: 17.4502846
Batch: 15,	Batch Loss: 10.9802127
Epoch: 005,  Loss:14.8502953,  
Batch: 05,	Batch Loss: 13.7640764
Batch: 10,	Batch Loss: 16.9271327
Batch: 15,	Batch Loss: 9.6954570
Epoch: 006,  Loss:13.4622220,  
Batch: 05,	Batch Loss: 14.3302119
Batch: 10,	Batch Loss: 11.4887091
Batch: 15,	Batch Loss: 9.2713945
Epoch: 007,  Loss:11.6967718,  
Batch: 05,	Batch Loss: 13.7278987
Batc

In [9]:
# save the model
torch.save(MWCNN_model, 'MWCNN_model_L1loss_16.pth')

In [4]:
MWCNN_model = torch.load('MWCNN_model_fft_16.pth')

In [8]:
# Testing process on test data.
mwcnn_psnr, mwcnn_mse = MWCNN_trainer.test(testloader)

print(f'PSNR: {mwcnn_psnr}, MSE: {mwcnn_mse}')

PSNR: 28.96020724376925, MSE: 2317197.6719563804


Load model

In [6]:
image_index = test_indices[0]
sample = image_dataset[image_index]
input_image, pred, output_image, score = MWCNN_trainer.predict(sample)

IndexError: too many indices for tensor of dimension 3